In [ ]:
import pyedflib
from pathlib import Path
import os
import numpy as np
from contextlib import ExitStack

In [ ]:
class SignalFile():
    def __init__(self, file_path: Path) -> None:
        self.filename = file_path.name
        self.file = pyedflib.EdfReader(self.filename)
        self.signals = self.file.signals_in_file
        self.labels = self.file.getSignalLabels()
        self.sample_rate = self.file.getSampleFrequencies()

    def load_signal_buffers(self):
        sigbufs = np.zeros((self.signals, self.file.getNSamples()[0]))

        for i in np.arange(self.signals):
            sigbufs[i, :] = self.file.readSignal(i)

        return list(zip(self.labels, sigbufs))
    
    def close_file(self):
        self.file.close()
    

def read_files_from_dir(directory: Path):
    extensions = ["edf", "bdf"]

    with ExitStack() as stack:
        files = [SignalFile(stack.enter_context(open(file.path, 'rb'))) for file in os.scandir(directory)
                 if file.is_file() and file.name.endswith(tuple(extensions))]

        return files
        

if __name__ == "__main__":
    directory = Path("data/edf/")
    files = read_files_from_dir(directory)
